__Preambule__

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

import numpy as np
np.corrcoef(*np.arange(10).reshape(2,-1))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
## importing stuff

import json
from collections import OrderedDict
from copy import deepcopy
from sys import exc_info
import os
import pickle
from time import sleep
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))

%load_ext autoreload
%autoreload 2
import pandas as pd
from islets.Recording import Recording, parse_leica
from islets.utils import get_filterSizes
from general_functions import td_nanfloor, td2str
from copy import deepcopy

from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

In [3]:
#%% other imports
def cellTransform(cellvalue, id=None, verbose=False):
    import numpy as np
    
    if issubclass(type(cellvalue),(bool,np.bool_)):
        if verbose:
            print ("creating a checklist")
        out = dcc.Checklist(id=id,
                             options=[{"label":"✔" if cellvalue else "✘","value":None}],
                             value=[None]*(1-int(cellvalue)),
                             labelStyle={
                                 "color":"green" if cellvalue else "red",
#                                  "padding":"3px"
                             }
                            )
#     elif issubclass(type(cellvalue),(dict,)):
#         out = dcc.Checklist(id=id,
#                              options=[{"label":(str(k).replace(" ","")+("✔" if cellvalue[k] else "✘")), "value":k} for k in cellvalue],
#                              value = [k for k in cellvalue if not cellvalue[k]],
# #                              style={"height":"10px",},
#                              labelStyle={
#                                  "width":"100px",
#                                  "display":"block"
#                                         }
#                             )
    elif issubclass(type(cellvalue),(str,int,float,np.int_)):
        out = cellvalue
    else:
        out = str(cellvalue)
    return html.Div(out,style={
                                 "font-family":"monospace",
                                 "font-size":"14px",
                             },)
#     if issubclass(type(value),str):

def mystyle(col):
    return {"border":'thin lightgrey solid',
            "text-align": "left" if "pickle" in col else None,
            "width": {
                "Series Durations":"120px",
                "Duration":"100px",
                "pickles":"150px",
                "exp":"150px",
                "series":"150px",
                "images":"150px",
                "line scan":"100px",
                     }.get(col),
           }

def generate_table(dataframe, max_rows=100):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            value = dataframe.iloc[i][col]
#             row.append(html.Td(value if issubclass(type(value),str) else dcc.Checklist(options=[{"label":"item"}]),
# #                                style=style
#                               ))
            row.append(html.Td(cellTransform(value, id=f"table-{col}-{i}"),
                               style=mystyle(col),
                              ))
        rows.append(html.Tr(row,
                            style={"border":'thin lightgrey solid'},
#                             border='thin lightgrey solid' 
                           ))

    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=mystyle(col)) for col in dataframe.columns],
                            style={"border":'thick lightgrey solid'},)] +
        # Body
        rows,
        style={
            "width": "1300px",
            "text-align":"right",
            "table-layout": "fixed"
        }
    ) 


def import_data(recordings,forceMetadataParse=False):
    status = []
    ilifs = 0
    for pathToRecording in tqdm(recordings):
        print ("#"*20, pathToRecording)
        try:
            rec = Recording(pathToRecording)
            if forceMetadataParse:
                rec.parse_metadata()
                rec.save_metadata()
        except:
            continue
        recType = "Nikon" if pathToRecording.endswith(".nd2") else "Leica"

        if recType=="Leica":
            sers = parse_leica(rec, index=True)
        else:
            sers = [(0,"all")]

        analysisFolder = os.path.join(rec.folder, rec.Experiment+"_analysis")
    #     if not os.path.isdir(analysisFolder):
    #         os.makedirs(analysisFolder)
#         rec.tag_linescans()
#         print (sers)
        for indices, ser in sers:
            md = pd.Series(dtype=object)
            md["path"] = pathToRecording
            md["exp"] = os.path.split(pathToRecording)[-1]
            md["series"] = ser
            index = indices[0]
            ser0 = "-".join(ser.split("-")[:-1]) if "-" in ser else ser
            assert rec.metadata.loc[index,"Name"]==ser0
#             singleLineScan = rec.metadata.loc[index,"SizeY"]>10
            try:
                rec.import_series(ser, onlyMeta=True,
#                                   isLineScan=rec.metadata.loc[index,"line scan"]=="single"
                                 )
            except:
                print (f"could not import {ser}")
                status += [md]
                continue
            for k,v in rec.Series[ser]["metadata"].items():
                md[k] = v
                
            saveDir = os.path.join(analysisFolder, ser)
            for k in ["bit depth", "Start time", "End time","Name"]: # , "individual Series"
                try:    del md[k]
                except: passs
            md["Series Durations"] = ["%s [%s]"%(r["Name"].replace("Series","S"), td2str(td_nanfloor(r["Duration"]))) for _,r in md["individual Series"].iterrows()]
            md["Series Durations"] = "\n".join(md["Series Durations"])
#             md["line scan"] = md["SizeY"]*md["pxSize"]<6 # treat also this as a line scan
            if md["line scan"] != "none":
                imagesDone = {}
                for name in md["individual Series"].Name:
                    imageName = os.path.join(saveDir, rec.Experiment+"_"+name+".png")
                    imagesDone[name] = os.path.isfile(imageName)
                md["images"] = imagesDone
                md["images done"] = all(imagesDone.values())
            else:
                movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
                md["movie"] = os.path.isfile(movieFilename)
                if md["movie"]:
                    md["movie size [MB]"] = os.path.getsize(movieFilename)/10**6
                pxSize = float(md.pxSize)
                if pxSize<.7: pxSize*=2
                fs = get_filterSizes(pxSize)
                pklsDone = {}
                imgsDone = {}
                for fsize in fs:
                    pickleFile = os.path.join(saveDir, ".".join(map(str,fsize))+"_rois.pkl")
                    imageFile = os.path.join(saveDir, ".image_"+".".join(map(str,fsize))+".png")
                    pickleThere = os.path.isfile(pickleFile)
                    imageThere = os.path.isfile(imageFile)
                    pklsDone[fsize] = pickleThere
                    imgsDone[fsize] = imageThere
                md["pickles"] = [pk if len(pk)>1 else pk for pk in pklsDone.keys()]
                md["(re)do pickles"] = all(pklsDone.values())
            del md["individual Series"]
            status += [dict(md.items())]
        ilifs +=1
    #     if ilifs>3:
    #         break

    return pd.DataFrame(status)

Enter Path to the folder you wish to process:

In [15]:
mainFolder = "/data/Sandra/2020/2020_11_05/"
# mainFolder = "/data/Marjan/MB2020_lifs_2_2/Arginine-photon-counting/"
# mainFolder = "/data/Sandra/2019/2019_09_03/"


recordings = []
for cur,ds,fs in os.walk(mainFolder):
    #### if you wish to restrict to only certain folders: ####
#     if "2020_11_05" not in cur: continue
    for f in fs:
        if not (f.endswith(".lif")):# or f.endswith(".nd2")):
            continue
        path = os.path.join(cur,f)
        recordings += [path]
recordings = sorted(recordings)

In [16]:
status_orig = import_data(recordings, forceMetadataParse=False)

#################### /data/Sandra/2020/2020_11_05/Experiment74a.lif
#################### /data/Sandra/2020/2020_11_05/Experiment74b.lif



In [18]:
rec = Recording("/data/Sandra/2020/2020_11_05/Experiment74a.lif")

In [19]:
rec.calc_gaps()

In [23]:
rec.metadata

,Name,SizeT,SizeX,SizeY,SizeZ,pxSize,pxUnit,bit depth,Frequency,Start time,End time,Duration,line scan,gap
0,Series001,23571,256,256,1,1.260,µm,uint8,19.8,2020-11-05 12:53:32,2020-11-05 13:13:23,0 days 00:19:51.802999496,none,NaN
1,Series002,7475,256,256,1,1.260,µm,uint8,19.8,2020-11-05 13:13:26,2020-11-05 13:19:43,0 days 00:06:17.937999725,none,3.0
2,Series003,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:15,NaT,NaT,none,1112.0
3,Series004,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:17,NaT,NaT,none,NaN
4,Series005,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:19,NaT,NaT,none,NaN
5,Series006,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:20,NaT,NaT,none,NaN
6,Series007,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:22,NaT,NaT,none,NaN
7,Series008,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:23,NaT,NaT,none,NaN
8,Series009,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:25,NaT,NaT,none,NaN
9,Series010,1,512,8000,1,0.391,µm,uint8,NaN,2020-11-05 13:38:26,NaT,NaT,none,NaN


In [27]:
rec.parse_metadata()

In [29]:
im = rec.xml.Image?

In [31]:
im = rec.xml.image(2)

In [43]:
pl = im.Pixels.Plane(0)

In [63]:
rec.xml

TypeError: 'type' object is not subscriptable

In [67]:
dir(rec.xml)

['Channel',
 'Detector',
 'Image',
 'Instrument',
 'Objective',
 'OriginalMetadata',
 'Pixels',
 'Plane',
 'Plate',
 'PlatesDucktype',
 'ROI',
 'ROIRef',
 'Rectangle',
 'StructuredAnnotations',
 'TiffData',
 'Union',
 'Well',
 'WellSample',
 'WellSampleDucktype',
 'WellsDucktype',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dom',
 'get_image_count',
 'get_ns',
 'get_roi_count',
 'image',
 'image_count',
 'instrument',
 'ns',
 'plates',
 'roi',
 'roi_count',
 'root_node',
 'set_image_count',
 'set_roi_count',
 'structured_annotations',
 'to_xml']

In [68]:
rec.xml.structured_annotations

{}

In [71]:
rec.rdr.read?

In [69]:
dir(rec.rdr)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close',
 'download',
 'init_reader',
 'metadata',
 'path',
 'rdr',
 'read',
 'stream',
 'using_temp_file']

In [24]:
rec.tag_linescans(indices=[2])

In [25]:
rec.metadata["line scan"].loc

0      none
1      none
2      none
3      none
4      none
5      none
6      none
7      none
8      none
9      none
10     none
11     none
12     none
13     none
14     none
15     none
16     none
17     none
18     none
19     none
20     none
21     none
22     none
23     none
24     none
25     none
26     none
27     none
28     none
29     none
30     none
31     none
32     none
33     none
34     none
35     none
36     none
37     none
38     none
39     none
40     none
41     none
42     none
43     none
44     none
45     none
46     none
47     none
48     none
49     none
50     none
51     none
52     none
53     none
54     none
55     none
56     none
57     none
58     none
59     none
60     none
61     none
62     none
63     none
64     none
65     none
66     none
67     none
68     none
69     none
70     none
71     none
72     none
73     none
74     none
75     none
76     none
77     none
78     none
79     none
80     none
81     none
82     none
83  

In [17]:
status_orig

,path,exp,series,SizeT,SizeX,SizeY,SizeZ,pxSize,pxUnit,Frequency,Duration,line scan,gap,frame_range,Series Durations,movie,movie size [MB],pickles,(re)do pickles
0,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series001-2,31046,256,256,1,1.260,µm,19.8,0 days 00:19:51.802999496,none,NaN,"(0, 31046)",S001 [19:51]\nS002 [6:17],True,2.690463,"[(5,), (6,), (5, 6), (5, 7)]",True
1,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series003,1,512,8000,1,0.391,µm,NaN,NaT,none,1112.0,"(0, 1)",S003 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
2,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series004,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S004 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
3,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series005,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S005 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
4,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series006,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S006 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
5,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series007,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S007 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
6,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series008,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S008 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
7,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series009,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S009 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
8,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series010,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S010 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False
9,/data/Sandra/2020/2020_11_05/Experiment74a.lif,Experiment74a.lif,Series011,1,512,8000,1,0.391,µm,NaN,NaT,none,NaN,"(0, 1)",S011 [N/A],False,NaN,"[(8,), (9,), (8, 9), (8, 11)]",False


In [6]:
# for _,row in status_orig.iterrows():
#     print (f"""/data/useful_notebooks/process_single.py --rec='{row.path}' --ser='{row.series}' --verbose""")

In [7]:
status = deepcopy(status_orig)
status.pxSize = status.pxSize.apply("{:.3f}".format).astype("str")
for c in status.columns:
    if "Size" in c and "px" not in c:
        status[c.replace("Size","")] = status[c]
        del status[c]

status["location"] = status.path.apply(lambda xi: os.path.split(xi)[0])
status["Freq"] = status["Frequency"].apply("{:.1f}".format).astype("str")
del status["Frequency"]
status["Duration"] = status["Duration"].apply(str).apply(lambda xi: xi.split()[-1].split(".")[0] )

firstCols = ["exp", "series","movie","Freq","Duration", "movie size [MB]", ]#,"images",]
status = status[[c for c in firstCols if c in status]+[c for c in status.columns if c not in firstCols]]
lastCols = ["path"]
status = status[[c for c in status.columns if c not in lastCols]+lastCols]

if "movie size [MB]" in status:
    status["movie size [MB]"] = status["movie size [MB]"].round(1)
for col in [
    "pxUnit",
    "frame_range",
    "gap",
    "pickles"
           ]:
    if col in status:
        del status[col]
# if "pickles" in status:
#     status.pickles = status_orig.pickles.apply(lambda xi: str(xi)[1:-1].replace(" ","") if "[" in str(xi) else "")

In [8]:
# status = status[~status[list("XYZT")].isna().all(1)]
# status = status[status.Z==1]

# status = status[~status[["movie","(re)do pickles"]].all(1)]

# status = status[~status.exp.apply(lambda xi: xi.endswith("nd2"))]

# status = status[status_orig.Duration>pd.Timedelta("30s")]

# status["line_scan"] = status["line scan"].astype(int)

status = status.sort_values(["exp","Y"])
status.index = range(len(status))

In [9]:
pd.set_option('display.max_rows', 500)

In [10]:
ix = status[status["line scan"]=="none"].query("T<10").index

In [11]:
status = status.drop(index=ix)

In [12]:
status.index = range(len(status))

In [13]:
# to hide
nshow = len(status)
htmltable = generate_table(status.iloc[:nshow,:-2], max_rows=nshow)
ixOrder = OrderedDict([(f"table-{col}-{i}",{"col":col,"index":i}) \
                       for i in range(nshow) for col in ["movie","(re)do pickles","images done"] \
                           if (col in status.columns) and (isinstance(status.loc[i,col],(bool,np.bool_)))
                      ])

app = JupyterDash(__name__, width=1000)

app.layout = html.Div([
    html.Div("Please check which of the following series you wish processed into movies/pickles. When done, please click on the button and follow instructions."),
    html.Br(),
    html.Button(id="save",children=["Prepare script"],n_clicks=0),
    html.Div(id="output"),
#     html.Button(id="check-all",children=["Check all"],n_clicks=0),
    html.Div(htmltable,style={"width":"1000px"}),
    html.Pre(id="marked",children="------------",
             style={
                "width": "700px",
                "height": "300px",
                'overflowX': 'scroll',
                'overflowY': 'scroll',
                "display": "block" if "srdjan" in os.getcwd() else "none",
            }),
],style={"font-family":"Arial"})

inputs = [Input(k,"value") for k in ixOrder]

@app.callback(
    Output("marked","children"),
    inputs
)
def see(*manyinputs):
    output = ""
    try:
        out = deepcopy(ixOrder)
        for k,v in zip(out, manyinputs):
            out[k]["value"] = v
        out = list(out.values())
        for el in out:
            el["value"] = bool(len(el["value"]))
            i = status.index[el["index"]]
            el["rec"] = status.loc[i,"path"]
            el["ser"] = status.loc[i,"series"]
            el["line scan"] = status.loc[i,"line scan"]
        out = pd.DataFrame(out)
        checklist_parse = out.query("col=='images done'").copy()
        if len(checklist_parse):
            del checklist_parse["col"], checklist_parse["index"], checklist_parse["value"]
        else: 
            checklist_parse = pd.DataFrame()
        out = out.query("col!='images done'")

        for (rec,ser),ddf in out.groupby(["rec","ser"]):
            el = {"rec":rec,"ser":ser}
            el["movie"] = ddf.query("col=='movie'")["value"].iloc[0]
            el["pickles"] = ddf.query("col=='(re)do pickles'")["value"].iloc[0]
            el["line scan"] = "none"
            checklist_parse = checklist_parse.append(el,ignore_index=True)
        ################################### good begin
        allrecs = checklist_parse.rec.unique()
#         output += "\n"+checklist_parse.__repr__()

        for rec in np.unique(allrecs):
            outFile = "/.".join(os.path.split(rec))+".out"
            output += f"rm {outFile}\n"
        for _,row in checklist_parse.iterrows():
            if row["line scan"]=="none":
                if not row[["movie","pickles"]].any():
                    continue
            outFile = "/.".join(os.path.split(row.rec))+".out"
            line = f'echo "" >> {outFile}\n'
            line += f'echo "" >> {outFile}\n'
            line += f'''echo "{'#'*50}" >> {outFile}\n'''
            line += f'''echo "###### processing of {row.ser} started at: $(date)" >> {outFile}\n'''
            line += f'''{{ time /data/useful_notebooks/process_single.py --recording="{row.rec}" --series="{row.ser}" --verbose'''

            if row["line scan"]!="none":
                line += " --line-scan="+row["line scan"]
            else:
                if not row.movie:
                    line += " --leave-movie"
                if not row.pickles:
                    line += " --leave-pickles"
            line += f" >> {outFile} ; }} 2>> {outFile}"
            output += line+"\n"
        ################################### good end
    except:
        output += str(exc_info())
    return output

# @app.callback(
#     [Output(k,"value") for k in ixOrder],
#     [Input("check-all","n_clicks")]
# )
# def checkall(nc):
#     if nc>0:
#         if nc%2:
#             return tuple([[None] for k in  ixOrder])
#         else:
#             return tuple([[] for k in  ixOrder])

@app.callback(
    Output("output","children"),
    [Input("save","n_clicks")],
    [State("marked","children")]
)
def prepare_script(n_clicks, text):
    if n_clicks>0:
        with open(os.path.expanduser("~/processing_script.sh"),"w") as f:
            f.write(text)
        return dcc.Markdown("""`processing_script.sh` prepared in your home folder.

Just open new terminal and run: `bash processing_script.sh` 

You can then close the window and wait until the processing is finished. 

_(For now, there is no reallyt good way how to follow progress, sorry. You can try the cell below...)_
""")

app._repr_html_() 
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')
# app

#### Hacky way to check progress

In [33]:
for exp,df in status.groupby("exp"):
    printExp = True
    for i,row in df.iterrows():
#         assert (status.loc[i].iloc[:2] == status_orig.loc[i,["exp","series"]]).all()
        try:
            images = [f".image_{'%i.%i'%fs if isinstance(fs,tuple) else fs}.png" for fs in status_orig.loc[i,"pickles"] ]
        except:
            continue
        serDir = f"{row.path}_analysis/{row.series}"
        if os.path.isdir(serDir):
            imDone = sum([el in images for el in os.listdir(serDir)])
        else:
            imDone = 0
        outPutFile = row.path.replace(row.exp, "."+row.exp+".out")
        percDone = 100/len(images)*imDone
    #     if percDone>=100: continue
        print ("%20s %13s  %3i%%"%(row.exp if printExp else "", row.series, percDone))#, outPutFile, os.path.isfile(outPutFile))
    #     if os.path.isfile(outPutFile):
    #         output = open(outPutFile).read()
    #         last = output#-1]
    #         last = "\n".join(["\t"+l for l in last.splitlines()])
    #         last += "\n"
    #         print(last)

    #     break
        printExp = False
    print ("-"*50)

--------------------------------------------------
